# **Bitcoin price prediction - Gradient Boosting Tree Regressor (Single Split)**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it


---


Description: perform model's train / validation with hyperparameter tuning and cross validation based on different methods of splitting the dataset.

# Global constants, dependencies, libraries and tools

In [1]:
# Main constants
LOCAL_RUNNING = True
SLOW_OPERATIONS = False # Decide whether or not to use operations that might slow down notebook execution
ROOT_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [2]:
if not LOCAL_RUNNING:
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(ROOT_DIR, force_remount=True)

    # Install Spark and related dependencies
    !pip install pyspark
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

## Import my utilities

In [69]:
# Set main dir
MAIN_DIR = ROOT_DIR + "" if LOCAL_RUNNING else ROOT_DIR + "/MyDrive/BDC/project"

# Utilities dir
UTILITIES_DIR = MAIN_DIR + "/utilities"

# Import my utilities
import sys
sys.path.append(UTILITIES_DIR)

from imports import *
import train_validation_utilities
from config import *

importlib.reload(train_validation_utilities)

<module 'train_validation_utilities' from 'D:\\Documents/Repository/BDC/project/utilities\\train_validation_utilities.py'>

## Core variables

In [4]:
# BS = Block Split
# WFS = Walk Forward Split
# SS = Single Split
SPLITTING_METHOD = SS 

# LR = LinearRegression 
# GLR = GeneralizedLinearRegression 
# RF = RandomForestRegressor 
# GBTR = GradientBoostingTreeRegressor
MODEL_NAME = GBTR

In [5]:
###################
# --- DATASET --- #
###################

# Datasets dirs
DATASET_OUTPUT_DIR = MAIN_DIR + "/datasets/output"

# Datasets paths
DATASET_TRAIN_VALID  = DATASET_OUTPUT_DIR + "/" + DATASET_TRAIN_VALID_NAME + ".parquet"

####################
# --- FEATURES --- #
####################

# Features dir
FEATURES_DIR = MAIN_DIR + "/features"

# Features paths
FEATURES_CORRELATION = FEATURES_DIR + "/" + FEATURES_CORRELATION_LABEL + ".json"
BASE_FEATURES = FEATURES_DIR + "/" + BASE_FEATURES_LABEL + ".json"
BASE_AND_MOST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_MOST_CORR_FEATURES_LABEL + ".json"
BASE_AND_LEAST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_LEAST_CORR_FEATURES_LABEL + ".json"

##################
# --- MODELS --- #
##################

# Model dir
MODELS_DIR = MAIN_DIR + "/models"

# Model path
MODEL = MODELS_DIR + "/" + MODEL_NAME

###################
# --- RESULTS --- #
###################

# Results dir
RESULTS_DIR = MAIN_DIR + "/results/" + SPLITTING_METHOD

# Results path
ALL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_all.csv"
REL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_rel.csv"

MODEL_ACCURACY_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_accuracy.csv"

In [6]:
# Importing useful libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pio.renderers.default = 'vscode+colab' # To correctly render plotly plots

# Create the pyspark session

In [7]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '109G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPricePrediction").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Loading dataset

In [8]:
# Load train / validation set into pyspark dataset objects
df = spark.read.load(DATASET_TRAIN_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [9]:
train_validation_utilities.dataset_info(df)

+-------------------+---+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+-------------------+------------------+--------------------+--------------------+------------------+--------------+--------------------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|          timestamp| id|     market-price|    opening-price|    highest-price|     lowest-price|    closing-price|  trade-volume-btc|   total-bitcoins|          market-cap|    trade-volume-usd|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|          hash-rate|        difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|n-transactions|estimated-transaction-volume-usd|       sma-5-days|   

Instead of considering the entire dataset, I will only consider the last two years from today's date in order to train / validate the model.

In [10]:
# Retrieve the last timestamp value
last_value = df.agg(last("timestamp")).collect()[0][0]

# Consider only the last two years as of today
split_date = last_value - relativedelta(years=2)

# Split the dataset based on the desired date and drop the "id" column
df = df[df['timestamp'] > split_date].drop(col("id"))

# Recompute id column
df = df.withColumn("id", F.row_number().over(Window.orderBy(F.monotonically_increasing_id()))-1)
# Rearranges columns
new_columns = ["timestamp", "id"] + [col for col in df.columns if col not in ["timestamp", "id", "next-market-price"]] + ["next-market-price"]
df = df.select(*new_columns)

In [11]:
train_validation_utilities.dataset_info(df)

+-------------------+---+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|          timestamp| id|      market-price|     opening-price|     highest-price|      lowest-price|     closing-price|  trade-volume-btc|   total-bitcoins|          market-cap|    trade-volume-usd|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|           hash-rate|          difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|    n-transactions|estimated-transaction-v

# Loading features

In [12]:
# Loading base features
with open(BASE_FEATURES, "r") as f:
    BASE_FEATURES = json.load(f)
print(BASE_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd']


In [13]:
# Loading currency and blockchain most correlated features
with open(BASE_AND_MOST_CORR_FEATURES, "r") as f:
    BASE_AND_MOST_CORR_FEATURES = json.load(f)
print(BASE_AND_MOST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days']


In [14]:
# Loading currency and blockchain least correlated features
with open(BASE_AND_LEAST_CORR_FEATURES, "r") as f:
    BASE_AND_LEAST_CORR_FEATURES = json.load(f)
print(BASE_AND_LEAST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'sma-100-days', 'transaction-fees-usd', 'n-unique-addresses', 'sma-50-days', 'n-transactions-total', 'blocks-size', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions']


# Model train / validation
In order to train and validate the model I'll try several approaches:
- `Default without normalization:` make predictions using the base model
- `Default with normalization:` like the previous one but features are normalized

Then the features that gave on average the most satisfactory results (for each model) are chosen and proceeded with:
- `Hyperparameter tuning:` finding the best parameters to use. 
- `Cross Validation:` validate the performance of the model with the chosen parameters (also here using Block split / Walk forward split)

If the final results are satisfactory, the model will be trained on the whole train / validation set and saved in order to make predictions on the test set.

For each approach the train / validation set will be split according to the chosen splitting method (in order to figure out which one works best for our problem). In this case the `Single time series split` method will be used: involves dividing the time series considering as validation set a narrow period of time and as train set everything that happened before this period, in such a way as to best benefit from the trend in the short term.

<img src="https://github.com/CorsiDanilo/bitcoin-price-prediction-with-pyspark/blob/main/notebooks/images/single-split.png?raw=1">

In [15]:
# Get splitting parameters based on the choosen splitting method
splitting_info = train_validation_utilities.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'single_split', 'split_label': 'months', 'split_value': 1}

## Default
The train / validation set will be splitted based on the splitting method chosen so that the model performance can be seen without any tuning by using different features (normalized and non)

In [16]:
# Get default parameters
params = train_validation_utilities.get_defaults_model_params(MODEL_NAME)
params

{'maxIter': [20], 'maxDepth': [5], 'stepSize': [0.1], 'seed': [42]}

### Without normalization

In [17]:
# Define model and features type
MODEL_TYPE = "default"
FEATURES_NORMALIZATION = False

In [18]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [19]:
# Make predictions by using base features
default_train_results_base_features, default_valid_results_base_features, default_train_pred_base_features, default_valid_pred_base_features = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [20]:
default_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,train,single_split,base_features,"(67104, 2976)","[20, 5, 0.1, 42]",423.123491,179033.488432,282.193835,0.008526,0.998957,0.998957,11.777838


In [21]:
default_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,single_split,base_features,"(67104, 2976)","[20, 5, 0.1, 42]",223.903748,50132.888391,193.859747,0.006566,0.707061,0.706667,11.777838


In [22]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [23]:
# Make predictions by using base and most additional correlated features
default_train_results_base_and_most_corr_features, default_valid_results_base_and_most_corr_features, default_train_pred_base_and_most_corr_features, default_valid_pred_base_and_most_corr_features = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [24]:
default_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,train,single_split,base_and_most_corr_features,"(67104, 2976)","[20, 5, 0.1, 42]",333.017441,110900.616269,245.057768,0.007619,0.999354,0.999354,8.784611


In [25]:
default_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,single_split,base_and_most_corr_features,"(67104, 2976)","[20, 5, 0.1, 42]",255.948567,65509.66899,221.841865,0.007486,0.617211,0.616695,8.784611


In [26]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [27]:
# Make predictions by using base and least additional correlated features
default_train_results_base_and_least_corr_features, default_valid_results_base_and_least_corr_features, default_train_pred_base_and_least_corr_features, default_valid_pred_base_and_least_corr_features = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [28]:
default_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,train,single_split,base_and_least_corr_features,"(67104, 2976)","[20, 5, 0.1, 42]",413.075986,170631.769883,273.738398,0.008256,0.999006,0.999006,7.113616


In [29]:
default_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,train,single_split,base_and_least_corr_features,"(67104, 2976)","[20, 5, 0.1, 42]",413.075986,170631.769883,273.738398,0.008256,0.999006,0.999006,7.113616


### With normalization

In [30]:
# Define model and features type
MODEL_TYPE = "default_norm"
FEATURES_NORMALIZATION = True

In [31]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [32]:
# Make predictions by using base features
default_norm_train_results_base_features, default_norm_valid_results_base_features, default_norm_train_pred_base_features, default_norm_valid_pred_base_features = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [33]:
default_norm_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,train,single_split,base_features_norm,"(67104, 2976)","[20, 5, 0.1, 42]",493.659728,243699.926879,318.648718,0.009384,0.99858,0.99858,5.51415


In [34]:
default_norm_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,valid,single_split,base_features_norm,"(67104, 2976)","[20, 5, 0.1, 42]",332.414793,110499.594695,280.714222,0.009504,0.354323,0.353454,5.51415


In [35]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [36]:
# Make predictions by using base and most additional correlated features
default_norm_train_results_base_and_most_corr_features, default_norm_valid_results_base_and_most_corr_features, default_norm_train_pred_base_and_most_corr_features, default_norm_valid_pred_base_and_most_corr_features = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [37]:
default_norm_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,train,single_split,base_and_most_corr_features_norm,"(67104, 2976)","[20, 5, 0.1, 42]",375.684226,141138.637557,272.977788,0.008498,0.999178,0.999178,5.3587


In [38]:
default_norm_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,valid,single_split,base_and_most_corr_features_norm,"(67104, 2976)","[20, 5, 0.1, 42]",282.334244,79712.625319,242.474319,0.008213,0.534219,0.533592,5.3587


In [39]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [40]:
# Make predictions by using base and least additional correlated features
default_norm_train_results_base_and_least_corr_features, default_norm_valid_results_base_and_least_corr_features, default_norm_train_pred_base_and_least_corr_features, default_norm_valid_pred_base_and_least_corr_features = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [41]:
default_norm_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,train,single_split,base_and_least_corr_features_norm,"(67104, 2976)","[20, 5, 0.1, 42]",574.39749,329932.476312,428.537301,0.013525,0.998078,0.998078,5.499143


In [42]:
default_norm_valid_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,valid,single_split,base_and_least_corr_features_norm,"(67104, 2976)","[20, 5, 0.1, 42]",1921.940099,3693853.742843,1850.084225,0.062404,-20.584107,-20.613167,5.499143


In [43]:
# Define model information and evaluators to show
model_info = ['Model', 'Type', 'Dataset', 'Splitting', 'Features', 'Parameters']
evaluator_lst = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2', 'Adjusted_R2', 'Time']

In [44]:
# Define the results to show
default_comparison_lst = [default_valid_results_base_features, default_valid_results_base_and_most_corr_features, default_valid_results_base_and_least_corr_features, default_norm_valid_results_base_features, default_norm_valid_results_base_and_most_corr_features, default_norm_valid_results_base_and_least_corr_features]

# Show the comparison table
default_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_comparison_lst])
default_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,single_split,base_features,"[20, 5, 0.1, 42]",223.903748,5.013289e+04,193.859747,0.006566,0.707061,0.706667,11.777838
0,GradientBoostingTreeRegressor,default,valid,single_split,base_and_most_corr_features,"[20, 5, 0.1, 42]",255.948567,6.550967e+04,221.841865,0.007486,0.617211,0.616695,8.784611
0,GradientBoostingTreeRegressor,default,valid,single_split,base_and_least_corr_features,"[20, 5, 0.1, 42]",201.127786,4.045239e+04,176.246971,0.005960,0.763627,0.763308,7.113616
0,GradientBoostingTreeRegressor,default_norm,valid,single_split,base_features_norm,"[20, 5, 0.1, 42]",332.414793,1.104996e+05,280.714222,0.009504,0.354323,0.353454,5.514150
0,GradientBoostingTreeRegressor,default_norm,valid,single_split,base_and_most_corr_features_norm,"[20, 5, 0.1, 42]",282.334244,7.971263e+04,242.474319,0.008213,0.534219,0.533592,5.358700
0,GradientBoostingTreeRegressor,default_norm,valid,single_split,base_and_least_corr_features_norm,"[20, 5, 0.1, 42]",1921.940099,3.693854e+06,1850.084225,0.062404,-20.584107,-20.613167,5.499143


In [48]:
# Save the best default model results and predicitons
best_default_results = default_valid_results_base_and_least_corr_features
best_default_predictions = default_valid_pred_base_and_least_corr_features
best_default_results

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,single_split,base_and_least_corr_features,"(67104, 2976)","[20, 5, 0.1, 42]",201.127786,40452.386189,176.246971,0.00596,0.763627,0.763308,7.113616


In [49]:
# Define the choosen features
FEATURES_NORMALIZATION = False
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

## Tuned
Once the features and execution method are selected, the model will undergo hyperparameter tuning and cross validation to find the best configuration.

In [50]:
# Using one of the multiple splitting methods to tune the model
SPLITTING_METHOD = "walk_forward_splits"

# Get splitting parameters based on the choosen splitting method
splitting_info = train_validation_utilities.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'walk_forward_splits',
 'min_obser': 20000,
 'sliding_window': 5000}

In [51]:
# Get model grid parameters
params = train_validation_utilities.get_model_grid_params(MODEL_NAME)
params

{'maxIter': [3, 5, 10, 20, 30],
 'maxDepth': [3, 5, 10],
 'stepSize': [0.1, 0.3, 0.5, 0.7],
 'seed': [42]}

### Hyperparameter tuning

In [52]:
# Select the type of feature to be used
MODEL_TYPE = "hyp_tuning"

In [53]:
# Perform hyperparameter tuning
hyp_res = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)
hyp_res

Split: [1/10]: 100%|██████████| 60/60 [04:03<00:00,  4.05s/it]


Best parameters chosen for split [1/10]: [30, 3, 0.5, 42]


Split: [2/10]: 100%|██████████| 60/60 [04:07<00:00,  4.12s/it]


Best parameters chosen for split [2/10]: [5, 3, 0.7, 42]


Split: [3/10]: 100%|██████████| 60/60 [04:02<00:00,  4.04s/it]


...

Split: [9/10]: 100%|██████████| 60/60 [04:02<00:00,  4.04s/it]


Best parameters chosen for split [9/10]: [3, 3, 0.5, 42]


Split: [10/10]: 100%|██████████| 60/60 [03:57<00:00,  3.96s/it]

Best parameters chosen for split [10/10]: [3, 5, 0.3, 42]


,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features,1,"(20000, 5000)","[30, 3, 0.5, 42]",392.281482,1.538848e+05,310.105206,0.007467,0.978022,0.978005,3.115470
1,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features,2,"(20000, 5000)","[5, 3, 0.7, 42]",7881.819485,6.212308e+07,6593.588001,0.254558,-1.776487,-1.778710,0.688413
2,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features,3,"(20000, 5000)","[20, 10, 0.7, 42]",1975.828453,3.903898e+06,1621.381851,0.072827,-0.897408,-0.898927,9.348781
3,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features,4,"(20000, 5000)","[30, 10, 0.1, 42]",291.436706,8.493535e+04,221.911012,0.011094,0.965563,0.965536,14.686364
4,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features,5,"(20000, 5000)","[3, 3, 0.7, 42]",1175.604789,1.382047e+06,816.809531,0.047112,0.461064,0.460632,0.559974
5,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features,6,"(20000, 5000)","[30, 5, 0.1, 42]",254.478647,6.475938e+04,184.728921,0.010721,0.911127,0.911055,4.779227
6,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features,7,"(20000, 5000)","[5, 5, 0.1, 42]",179.414003,3.218938e+04,128.280619,0.005525,0.963785,0.963756,0.964958
7,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features,8,"(20000, 5000)","[30, 3, 0.7, 42]",3094.058657,9.573199e+06,2759.793977,0.096833,-0.603556,-0.604841,3.169839
8,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features,9,"(20000, 5000)","[3, 3, 0.5, 42]",455.482126,2.074640e+05,359.283478,0.013416,0.771913,0.771731,0.501038
9,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features,10,"(20000, 5000)","[3, 5, 0.3, 42]",242.649474,5.887877e+04,202.126432,0.006721,0.808220,0.808067,0.652425


---
Since during this stage will be used the Block split / Walk forward split method of the dataset I compute a score for each parameter chosen by each split, assigning weights based on:
   * Their `frequency` for each split (if the same parameters are chosen from several splits, these will have greater weight) 
   * The `split` they belong to (the closer the split is to today's date the more weight they will have)
   * Their `RMSE value` for each split (the lower this is, the more weight they will have)
   
   Then, the overall score will be calculated by putting together these three weights for each parameter and the one with the best score will be the chosen parameter.

In [54]:
# Show parameters score
grouped_scores, best_params = train_validation_utilities.choose_best_params(hyp_res)
grouped_scores

,Split weight,RMSE weight,Frequency weight,Final score
Parameters,,,,
"(3, 5, 0.3, 42)",1.0,0.969214,1.0,0.969214
"(3, 3, 0.5, 42)",0.9,0.942211,1.0,0.847990
"(5, 5, 0.1, 42)",0.7,0.977237,1.0,0.684066
"(30, 5, 0.1, 42)",0.6,0.967713,1.0,0.580628
"(30, 3, 0.7, 42)",0.8,0.607444,1.0,0.485955
"(3, 3, 0.7, 42)",0.5,0.850846,1.0,0.425423
"(30, 10, 0.1, 42)",0.4,0.963024,1.0,0.385210
"(20, 10, 0.7, 42)",0.3,0.749318,1.0,0.224795
"(30, 3, 0.5, 42)",0.1,0.950230,1.0,0.095023


In [55]:
# Print best parameters
print(f"Best parameters: {best_params}")

Best parameters: (3, 5, 0.3, 42)


### Cross validation

In [56]:
MODEL_TYPE = "cross_val"

In [57]:
# Get tuned parameters
params = train_validation_utilities.get_best_model_params(best_params, MODEL_NAME)
params

{'maxIter': [3], 'maxDepth': [5], 'stepSize': [0.3], 'seed': [42]}

In [58]:
# Perform cross validation
cv_train_result, cv_valid_result, cv_train_pred, cv_valid_pred = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [10/10]: 100%|██████████| 1/1 [00:01<00:00,  1.70s/it]


In [59]:
cv_train_result

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_and_least_corr_features,1,"(20000, 5000)","[3, 5, 0.3, 42]",303.772169,92277.530933,240.214938,0.005030,0.998572,0.998572,0.938944
1,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_and_least_corr_features,2,"(20000, 5000)","[3, 5, 0.3, 42]",314.268954,98764.975365,245.609198,0.005236,0.998669,0.998668,0.891088
2,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_and_least_corr_features,3,"(20000, 5000)","[3, 5, 0.3, 42]",344.621092,118763.697156,262.340875,0.006798,0.998027,0.998027,0.885748
3,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_and_least_corr_features,4,"(20000, 5000)","[3, 5, 0.3, 42]",208.806170,43600.016783,162.246221,0.005152,0.999430,0.999429,0.878238
4,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_and_least_corr_features,5,"(20000, 5000)","[3, 5, 0.3, 42]",244.909102,59980.468474,174.832799,0.006198,0.999261,0.999261,0.888930
5,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_and_least_corr_features,6,"(20000, 5000)","[3, 5, 0.3, 42]",159.041860,25294.313220,115.961350,0.005056,0.998974,0.998974,0.894358
6,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_and_least_corr_features,7,"(20000, 5000)","[3, 5, 0.3, 42]",91.109684,8300.974478,70.577808,0.003576,0.998345,0.998344,0.878452
7,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_and_least_corr_features,8,"(20000, 5000)","[3, 5, 0.3, 42]",82.240337,6763.473058,62.698565,0.003163,0.998921,0.998921,0.890170
8,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_and_least_corr_features,9,"(20000, 5000)","[3, 5, 0.3, 42]",125.418669,15729.842495,89.189296,0.004176,0.999154,0.999154,0.872867
9,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_and_least_corr_features,10,"(20000, 5000)","[3, 5, 0.3, 42]",126.359507,15966.725022,98.500465,0.004224,0.999192,0.999191,0.871812


In [60]:
cv_valid_result

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_and_least_corr_features,1,"(20000, 5000)","[3, 5, 0.3, 42]",444.020488,1.971542e+05,370.119325,0.008928,0.971842,0.971820,0.938944
1,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_and_least_corr_features,2,"(20000, 5000)","[3, 5, 0.3, 42]",8142.835765,6.630577e+07,6873.473156,0.264748,-1.963425,-1.965798,0.891088
2,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_and_least_corr_features,3,"(20000, 5000)","[3, 5, 0.3, 42]",2037.847314,4.152822e+06,1684.439588,0.075497,-1.018392,-1.020008,0.885748
3,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_and_least_corr_features,4,"(20000, 5000)","[3, 5, 0.3, 42]",416.453424,1.734335e+05,346.305847,0.017234,0.929682,0.929625,0.878238
4,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_and_least_corr_features,5,"(20000, 5000)","[3, 5, 0.3, 42]",1324.557568,1.754453e+06,795.521305,0.047234,0.315842,0.315294,0.888930
5,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_and_least_corr_features,6,"(20000, 5000)","[3, 5, 0.3, 42]",310.469396,9.639125e+04,244.521901,0.014231,0.867716,0.867610,0.894358
6,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_and_least_corr_features,7,"(20000, 5000)","[3, 5, 0.3, 42]",194.530410,3.784208e+04,141.154218,0.006092,0.957426,0.957392,0.878452
7,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_and_least_corr_features,8,"(20000, 5000)","[3, 5, 0.3, 42]",3569.863884,1.274393e+07,3181.140567,0.111131,-1.134669,-1.136378,0.890170
8,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_and_least_corr_features,9,"(20000, 5000)","[3, 5, 0.3, 42]",633.568303,4.014088e+05,448.280771,0.016804,0.558690,0.558336,0.872867
9,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_and_least_corr_features,10,"(20000, 5000)","[3, 5, 0.3, 42]",242.649474,5.887877e+04,202.126432,0.006721,0.808220,0.808067,0.871812


Let's see if the situation has improved by validating the model with the newly found parameters.

In [61]:
# Using single split method to validate the model
SPLITTING_METHOD = "single_split"

# Get splitting parameters based on the choosen splitting method
splitting_info = train_validation_utilities.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'single_split', 'split_label': 'months', 'split_value': 1}

In [62]:
MODEL_TYPE = "tuned"

In [63]:
# Validate the model with the best parameters
tuned_train_results, tuned_valid_results, tuned_train_pred, tuned_valid_pred = train_validation_utilities.single_split(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [64]:
tuned_train_results

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,tuned,train,single_split,base_and_least_corr_features,"(67104, 2976)","[3, 5, 0.3, 42]",469.465671,220398.016357,330.388245,0.010111,0.998716,0.998716,1.123544


In [65]:
tuned_valid_results

,Model,Type,Dataset,Splitting,Features,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,tuned,valid,single_split,base_and_least_corr_features,"(67104, 2976)","[3, 5, 0.3, 42]",181.871695,33077.313458,151.631595,0.005134,0.806721,0.806461,1.123544


In [66]:
# Define the results to show
tuned_comparison_lst = [cv_valid_result, tuned_valid_results]

# Show the comparison table
tuned_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in tuned_comparison_lst])
tuned_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_and_least_corr_features,"[3, 5, 0.3, 42]",1731.679603,8.592209e+06,1428.708311,0.056862,0.129293,0.128596,0.889061
0,GradientBoostingTreeRegressor,tuned,valid,single_split,base_and_least_corr_features,"[3, 5, 0.3, 42]",181.871695,3.307731e+04,151.631595,0.005134,0.806721,0.806461,1.123544


## Trained
At this point it is possible to train the final model over the entire train / validation set in order to use it in the final step, that is, to make predictions about the data contained in the test set.

In [70]:
MODEL_TYPE = "final"

In [71]:
# Train the final model on the whole train / validation set
final_train_results, final_model, final_train_pred = train_validation_utilities.evaluate_trained_model(df, params, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

In [72]:
final_train_results

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,final,train,whole_train_valid,base_and_least_corr_features,"[3, 5, 0.3, 42]",479.156867,229591.303652,309.509288,0.009205,0.998606,0.998606,1.587729


# Comparison table
Visualization of model performance at various stages of train / validation

In [73]:
# Concatenate final results into Pandas dataset
final_comparison_lst_df = pd.DataFrame(pd.concat([default_comparison_lst_df, tuned_comparison_lst_df, final_train_results], ignore_index=True))
final_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,single_split,base_features,"[20, 5, 0.1, 42]",223.903748,5.013289e+04,193.859747,0.006566,0.707061,0.706667,11.777838
1,GradientBoostingTreeRegressor,default,valid,single_split,base_and_most_corr_features,"[20, 5, 0.1, 42]",255.948567,6.550967e+04,221.841865,0.007486,0.617211,0.616695,8.784611
2,GradientBoostingTreeRegressor,default,valid,single_split,base_and_least_corr_features,"[20, 5, 0.1, 42]",201.127786,4.045239e+04,176.246971,0.005960,0.763627,0.763308,7.113616
3,GradientBoostingTreeRegressor,default_norm,valid,single_split,base_features_norm,"[20, 5, 0.1, 42]",332.414793,1.104996e+05,280.714222,0.009504,0.354323,0.353454,5.514150
4,GradientBoostingTreeRegressor,default_norm,valid,single_split,base_and_most_corr_features_norm,"[20, 5, 0.1, 42]",282.334244,7.971263e+04,242.474319,0.008213,0.534219,0.533592,5.358700
5,GradientBoostingTreeRegressor,default_norm,valid,single_split,base_and_least_corr_features_norm,"[20, 5, 0.1, 42]",1921.940099,3.693854e+06,1850.084225,0.062404,-20.584107,-20.613167,5.499143
6,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_and_least_corr_features,"[3, 5, 0.3, 42]",1731.679603,8.592209e+06,1428.708311,0.056862,0.129293,0.128596,0.889061
7,GradientBoostingTreeRegressor,tuned,valid,single_split,base_and_least_corr_features,"[3, 5, 0.3, 42]",181.871695,3.307731e+04,151.631595,0.005134,0.806721,0.806461,1.123544
8,GradientBoostingTreeRegressor,final,train,whole_train_valid,base_and_least_corr_features,"[3, 5, 0.3, 42]",479.156867,2.295913e+05,309.509288,0.009205,0.998606,0.998606,1.587729


# Model accuracy

Since predicting the price accurately is very difficult I also saw how good the models are at predicting whether the price will go up or down in this way:

For each prediction let's consider the actual market-price, next-market-price and our predicted next-market-price (prediction).
I compute whether the current prediction is correct (1) or not (0):

$$ 
prediction\_is\_correct
= 
\begin{cases}
0 \text{ if [(market-price > next-market-price) and (market-price < prediction)] or [(market-price < next-market-price) and (market-price > prediction)]} \\
1 \text{ if [(market-price > next-market-price) and (market-price > prediction)] or [(market-price < next-market-price) and (market-price < prediction)]}
\end{cases}
$$

After that I count the number of correct prediction:
$$ 
correct\_predictions
= 
\sum_{i=0}^{total\_rows} prediction\_is\_correct
$$

Finally I compute the percentage of accuracy of the model:
$$
\\ 
accuracy 
= 
(correct\_predictions / total\_rows) 
* 100
$$

In [74]:
# Convert the pandas dataset to a PySpark dataset
best_default_pred_spark = spark.createDataFrame(best_default_predictions)
validated_pred_spark = spark.createDataFrame(tuned_valid_pred)

# Compute model accuracy
default_accuracy = train_validation_utilities.model_accuracy(best_default_pred_spark)
validated_accuracy = train_validation_utilities.model_accuracy(validated_pred_spark)

# Shows whether features are normalised or not
if FEATURES_NORMALIZATION:
    NEW_CHOSEN_FEATURES_LABEL = CHOSEN_FEATURES_LABEL + "_norm"
    CHOSEN_FEATURES_LABEL = NEW_CHOSEN_FEATURES_LABEL

# Saving accuracy data into dataframe
accuracy_data = {
    'Model': MODEL_NAME,
    'Features': CHOSEN_FEATURES_LABEL,
    'Splitting': SPLITTING_METHOD,
    'Accuracy (default)': default_accuracy,
    'Accuracy (tuned)': validated_accuracy
}
accuracy_data_df = pd.DataFrame(accuracy_data, index=['Model'])

print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (default): {default_accuracy:.2f}%")
print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (tuned): {validated_accuracy:.2f}%")

Percentage of correct predictions for GradientBoostingTreeRegressor with base_and_least_corr_features and single_split (default): 54.54%
Percentage of correct predictions for GradientBoostingTreeRegressor with base_and_least_corr_features and single_split (tuned): 55.78%


In [75]:
# Concatenate default and tuned results
default_tuned_results = [best_default_results, tuned_valid_results]
default_tuned_results_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_tuned_results])
default_tuned_results_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,single_split,base_and_least_corr_features,"[20, 5, 0.1, 42]",201.127786,40452.386189,176.246971,0.005960,0.763627,0.763308,7.113616
0,GradientBoostingTreeRegressor,tuned,valid,single_split,base_and_least_corr_features,"[3, 5, 0.3, 42]",181.871695,33077.313458,151.631595,0.005134,0.806721,0.806461,1.123544


# Saving final results

In [76]:
# Save all final comparison results
final_comparison_lst_df.to_csv(ALL_MODEL_RESULTS, index=False)

In [77]:
# Save relevant results (default and tuned results)
default_tuned_results_df.to_csv(REL_MODEL_RESULTS, index=False)

In [78]:
# Saving accuracy results
accuracy_data_df.to_csv(MODEL_ACCURACY_RESULTS, index=False)

In [79]:
# Saving final model
final_model.write().overwrite().save(MAIN_DIR + '/models/' + MODEL_NAME)

In [80]:
# Export notebook in html format (remember to save the notebook and change the model name)
if LOCAL_RUNNING:
    !jupyter nbconvert --to html 5-single-split_{MODEL_NAME}.ipynb --output 5-single-split_{MODEL_NAME} --output-dir='./exports'

d:\Documents\Repository\BDC\project\.bdc\Lib\site-packages\traitlets\traitlets.py:2548: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use './exports' instead of "'./exports'" if you require traitlets >=5.
  warn(
[NbConvertApp] Converting notebook 5-single-split_GradientBoostingTreeRegressor.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 1 image(s).
[NbConvertApp] Writing 462094 bytes to exports\5-single-split_GradientBoostingTreeRegressor.html
